In [1]:
%pip install ultralytics > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ultralytics
import torch
import os
import json
import yaml
import pandas as pd
import shutil
import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt

In [3]:
X_train = np.load("/kaggle/input/sofavision/X_train.npy")
y_train = np.load("/kaggle/input/sofavision/y_train.npy")
X_test = np.load("/kaggle/input/sofavision/X_test.npy")

In [4]:
X_train.shape

(245, 224, 224, 3)

In [5]:
os.mkdir('/kaggle/working/dataset')
os.mkdir('/kaggle/working/dataset/images')
os.mkdir('/kaggle/working/dataset/labels')
os.mkdir('/kaggle/working/dataset/images/train')
os.mkdir('/kaggle/working/dataset/images/val')
os.mkdir('/kaggle/working/dataset/labels/train')
os.mkdir('/kaggle/working/dataset/labels/val')

In [6]:
for idx, arr in enumerate(X_train):
    im = Image.fromarray(arr)
    im.save(f'/kaggle/working/dataset/images/train/img{idx}.jpg')
#     im.save(f'/kaggle/working/dataset/images/val/img{idx}.jpg')
    

In [7]:
for idx, mask in enumerate(y_train):
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    with open(f'/kaggle/working/dataset/labels/train/img{idx}.txt', 'w') as f:
        for contour in contours:
            if contour.shape[0] > 20:
                
                f.write('0')
                for pair in contour:
                    x = pair[0][0] / 224
                    y = pair[0][1] / 224
                    f.write(f' {x} {y}')
                
                f.write('\n')

In [8]:
data = {
    'path': '/kaggle/working/dataset',
    'train': 'images/train',
    'val': f'images/train',
    "names": {
          0: 'sofa'
    }
}

with open('config.yaml', 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

In [9]:
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU is not available, using CPU instead")
    device = torch.device("cpu")

GPU is available


In [10]:
model = ultralytics.YOLO('yolov8s-seg.pt')

model.to(device)

100%|██████████| 22.8M/22.8M [00:00<00:00, 52.1MB/s]


YOLO(
  (model): SegmentationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_runnin

In [11]:
# Currently not using wandb
os.environ['WANDB_DISABLED'] = 'true'

In [12]:
model.train(data="config.yaml", epochs=20, imgsz=224, device='0')

Ultralytics YOLOv8.2.10 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

100%|██████████| 755k/755k [00:00<00:00, 4.31MB/s]
2024-05-06 13:45:25,568	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-06 13:45:26,377	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-06 13:45:28.533602: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 13:45:28.533697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 13:45:28.662823: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS facto

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

wandb: Tracking run with wandb version 0.16.6
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...


20 epochs completed in 0.031 hours.
Optimizer stripped from runs/segment/train/weights/last.pt, 23.8MB
Optimizer stripped from runs/segment/train/weights/best.pt, 23.8MB

Validating runs/segment/train/weights/best.pt...
Ultralytics YOLOv8.2.10 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
YOLOv8s-seg summary (fused): 195 layers, 11779987 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.89it/s]


                   all        245        250      0.995          1      0.995      0.916      0.995          1      0.995      0.963
Speed: 0.0ms preprocess, 1.2ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/segment/train


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▂▄▅▆▇██▇▇▆▆▅▅▄▄▃▃▂▂▁
wandb:                  lr/pg1 ▂▄▅▆▇██▇▇▆▆▅▅▄▄▃▃▂▂▁
wandb:                  lr/pg2 ▂▄▅▆▇██▇▇▆▆▅▅▄▄▃▃▂▂▁
wandb:        metrics/mAP50(B) ▇▃▁▅▄▄▅▇████████████
wandb:        metrics/mAP50(M) ▇▂▁▅▃▄▅▇████████████
wandb:     metrics/mAP50-95(B) ▇▃▁▂▁▁▃▆▆▇▆▆▇▇▇█████
wandb:     metrics/mAP50-95(M) ▆▁▁▂▂▁▄▆▆▆▆▆▇▇██████
wandb:    metrics/precision(B) ▇▃▁▄▂▃▄▇▇█▇▇▆███████
wandb:    metrics/precision(M) ▇▂▁▃▁▃▄▇▇█▇▇▆███████
wandb:       metrics/recall(B) █▃▄▄▅▁▁▄▇█▆▇▆█▇█████
wandb:       metrics/recall(M) █▂▅▅▆▁▂▅▇█▇▇▇███████
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▅▆▆▆▆▇▅▅▅▆▄▄▃▃▂▂▂▁▁
wandb:          train/cls_loss █▃▃▃▂▂▃▂▂▂▄▃▂▂▂▁▁▁▁▁
wandb:          train/dfl_loss ▇▄▅▅▅▅▆▄▄▄█▆▆▄▃▃▃▂▁▁
wandb:          train/seg_loss █▃▃▃▂▃▃▂▂▂▄▂▂▂▂▂▁▁▁▁
wandb:

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cd6cf0bb790>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [13]:
predicted_masks = []

for arr in X_test:
    pred = model(arr)[0]
    predicted_mask = pred.masks.data[0].cpu().detach().numpy()
    predicted_masks.append(predicted_mask)


0: 224x224 1 sofa, 11.1ms
Speed: 1.0ms preprocess, 11.1ms inference, 3.8ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 1 sofa, 11.0ms
Speed: 0.8ms preprocess, 11.0ms inference, 2.9ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 1 sofa, 10.7ms
Speed: 0.9ms preprocess, 10.7ms inference, 2.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 1 sofa, 10.2ms
Speed: 0.8ms preprocess, 10.2ms inference, 3.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 1 sofa, 10.0ms
Speed: 0.7ms preprocess, 10.0ms inference, 2.8ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 1 sofa, 10.0ms
Speed: 0.9ms preprocess, 10.0ms inference, 2.6ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 2 sofas, 10.2ms
Speed: 0.7ms preprocess, 10.2ms inference, 2.6ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 1 sofa, 10.3ms
Speed: 0.7ms preprocess, 10.3ms inference, 2.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224

In [14]:
def encode_mask(mask):
  pixels = mask.flatten()
  #Possible for some masks to be in the corners
  pixels = np.concatenate([[0], pixels, [0]])
  runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
  runs[1::2] = runs[1::2] - runs[:-1:2]

  return ' '.join(str(x) for x in runs)

def submissions_file(predictions_npy, output_file):
  encoded_predictions = [(str(i) + ',' + encode_mask(mask)) for i, mask in enumerate(predictions_npy)]

  with open(output_file + '.csv', 'w') as submission_file:
    # Might need to change headers, depends on our submission guidelines
    submission_file.write('ID,Prediction\n')
    for string in encoded_predictions:
      submission_file.write(string + '\n')


In [15]:
submissions_file(predicted_masks, 'sofa_submission')